# Experiment 3: Baseline + Forecast + Context Features

This notebook extends Experiment 2 by adding context asset forecast features from market-wide ETFs (SPY, QQQ).

**New Features Added:**
- Context asset forecasts (SPY, QQQ) at multiple horizons
- Relative features: asset vs market forecast comparisons
- Market regime indicators from context forecasts

**Hypothesis:**
Market context provides additional signal for individual asset prediction. When the market is expected to rally, individual assets may behave differently.

**Prerequisites:**
- Run Experiment 0/1 first to generate labeled dataset

## Configuration

In [ ]:
# ============================================================
# CONFIGURATION
# ============================================================

CONFIG = {
    # Symbols to train on
    "symbols_to_train": ["SPY"],
    
    # Context symbols for market features
    "context_symbols": ["SPY"],  # Can add "QQQ" for more context
    
    # Data limits
    "max_rows_per_symbol": 6500,
    
    # Label parameters
    "label_col": "label",
    "vertical_barrier_bars": 26,
    
    # Split parameters
    "embargo_bars": 26,
    "tune_window": 260,
    "test_window": 520,
    "min_train_size": 2000,
    
    # AutoGluon Tabular
    "time_limit_sec": 1200,
    "presets": "best_quality",
    
    # Time Series
    "ts_prediction_length": 26,
    "ts_presets": "chronos_small",
    "ts_train_lookback_years": 5.0,
    
    # Feature parameters
    "feature_set": "small",
    "forecast_prefix": "fc_",
    "include_relative": True,  # Include relative features (asset vs context)
    
    # Reproducibility
    "random_seed": 42,
    
    # Force options
    "force_data_refresh": False,
    "force_ts_retrain": False,
    "force_forecast_regenerate": False,
}

print("Configuration:")
for k, v in CONFIG.items():
    print(f"  {k}: {v}")

## Setup

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Install dependencies
print("Installing dependencies...")
!pip install -q autogluon.tabular[tabarena] || pip install -q autogluon.tabular[all]
!pip install -q autogluon.timeseries[chronos-openvino]
!pip install -q pandas numpy pyarrow scikit-learn pytz alpaca-py
print("\nInstallation complete!")

In [ ]:
# Clone/update repository
import os

REPO_URL = "https://github.com/mh122333/ETF-Dual-Foundation-Project-CC-Version.git"
REPO_DIR = "/content/ETF-Dual-Foundation-Project-CC-Version"
BRANCH = "claude/build-pipeline-sanity-exp-iVs65"

if os.path.exists(REPO_DIR):
    print("Repository exists, updating...")
    %cd {REPO_DIR}
    !git fetch origin
    !git checkout {BRANCH}
    !git pull origin {BRANCH}
else:
    print("Cloning repository...")
    !git clone {REPO_URL} {REPO_DIR}
    %cd {REPO_DIR}
    !git checkout {BRANCH}

print(f"\nOn branch: {BRANCH}")

In [ ]:
# Add src to path and set random seeds
import sys
import random
import numpy as np

sys.path.insert(0, '/content/ETF-Dual-Foundation-Project-CC-Version/src')

random.seed(CONFIG["random_seed"])
np.random.seed(CONFIG["random_seed"])

print(f"Random seed set to: {CONFIG['random_seed']}")

In [ ]:
# Imports
from datetime import datetime
from pathlib import Path
import json

import pandas as pd
import numpy as np

# Project imports
from etf_pipeline.utils.paths import get_drive_paths, ensure_dirs, get_labeled_dataset_path
from etf_pipeline.splits.purged_walkforward import (
    create_single_split,
    apply_split_to_dataframe,
    validate_split_no_leakage,
)
from etf_pipeline.models.tabular_baseline import (
    train_tabular_baseline,
    predict_tabular,
)
from etf_pipeline.metrics.classification import (
    compute_all_metrics,
    save_metrics,
    print_metrics_summary,
)

# Time series imports
from etf_pipeline.timeseries.dataset import prepare_ts_training_data
from etf_pipeline.timeseries.train import load_or_train_timeseries_predictor
from etf_pipeline.timeseries.rolling_predict import load_or_generate_forecasts

# Feature imports
from etf_pipeline.features.forecast_features import (
    merge_forecast_features,
    get_forecast_feature_names,
    FEATURE_SET_CONFIGS,
)
from etf_pipeline.features.context_features import (
    add_context_features,
    compute_relative_features,
)
from etf_pipeline.features.baseline import get_feature_columns

print("Imports successful!")

In [ ]:
# Create output directories
paths = ensure_dirs()
print("Output directories:")
for name, path in paths.items():
    print(f"  {name}: {path}")

# Generate run ID
run_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
symbols_str = "_".join(CONFIG["symbols_to_train"])
RUN_ID = f"exp3_{symbols_str}_{run_timestamp}"
print(f"\nRun ID: {RUN_ID}")

## 1. Load Data

In [ ]:
# Load labeled dataset
labeled_dataset_path = get_labeled_dataset_path()
print(f"Loading labeled dataset from: {labeled_dataset_path}")

if not labeled_dataset_path.exists():
    raise FileNotFoundError(f"Labeled dataset not found. Run Experiment 0/1 first.")

full_df = pd.read_parquet(labeled_dataset_path)
print(f"Loaded {len(full_df)} rows")

In [ ]:
# Load raw bars
bars_path = paths["raw"] / "bars_30min.parquet"

if bars_path.exists():
    print(f"Loading cached bars from: {bars_path}")
    bars_df = pd.read_parquet(bars_path)
else:
    print("Fetching bars from Alpaca...")
    from google.colab import userdata
    from alpaca.data.historical import StockHistoricalDataClient
    import pytz
    from etf_pipeline.data.alpaca import load_all_symbols
    
    api_key = userdata.get("PAPER_KEY")
    api_secret = userdata.get("PAPER_SEC")
    client = StockHistoricalDataClient(api_key, api_secret)
    
    eastern = pytz.timezone("US/Eastern")
    start = eastern.localize(datetime(2019, 1, 1))
    end = eastern.localize(datetime(2025, 12, 31))
    
    # Include context symbols
    all_symbols = list(set(CONFIG["symbols_to_train"] + CONFIG["context_symbols"] + ["SPY", "QQQ"]))
    bars_df = load_all_symbols(client, all_symbols, start, end, cache=True)
    bars_df.to_parquet(bars_path)
    print(f"Saved bars to: {bars_path}")

print(f"Bars shape: {bars_df.shape}")

## 2. Generate Forecasts for Target and Context Symbols

In [ ]:
# Define horizons
feature_set_config = FEATURE_SET_CONFIGS[CONFIG["feature_set"]]
horizons = feature_set_config["horizons"]
print(f"Feature set: {CONFIG['feature_set']}")
print(f"Horizons: {horizons}")

# All symbols that need forecasts (target + context)
all_forecast_symbols = list(set(CONFIG["symbols_to_train"] + CONFIG["context_symbols"]))
print(f"\nSymbols needing forecasts: {all_forecast_symbols}")

In [ ]:
# Generate forecasts for all symbols
all_forecasts = {}

# Get decision timestamps from the first target symbol
first_target = CONFIG["symbols_to_train"][0]
if isinstance(full_df.index, pd.MultiIndex):
    target_labeled = full_df.loc[first_target].copy()
else:
    target_labeled = full_df[full_df["symbol"] == first_target].copy()
target_labeled = target_labeled.sort_index()

max_rows = CONFIG["max_rows_per_symbol"]
if max_rows and len(target_labeled) > max_rows:
    target_labeled = target_labeled.iloc[-max_rows:]

decision_timestamps = target_labeled.index.tolist()
print(f"Decision timestamps: {len(decision_timestamps)}")

for symbol in all_forecast_symbols:
    print(f"\n{'=' * 60}")
    print(f"GENERATING FORECASTS FOR: {symbol}")
    print(f"{'=' * 60}")
    
    # Get symbol bars
    if isinstance(bars_df.index, pd.MultiIndex):
        symbol_bars = bars_df.loc[symbol].copy()
    else:
        symbol_bars = bars_df[bars_df["symbol"] == symbol].copy()
    symbol_bars = symbol_bars.sort_index()
    
    # Model path
    ts_model_path = paths["models"] / "ts" / symbol / f"pred_len_{CONFIG['ts_prediction_length']}"
    ts_model_path.mkdir(parents=True, exist_ok=True)
    
    # Prepare training data
    first_decision = decision_timestamps[0]
    train_data = prepare_ts_training_data(
        bars_df=symbol_bars,
        symbols=[symbol],
        train_end_timestamp=first_decision,
        lookback_years=CONFIG["ts_train_lookback_years"],
    )
    
    # Train/load predictor
    ts_predictor = load_or_train_timeseries_predictor(
        train_data=train_data,
        model_path=ts_model_path,
        prediction_length=CONFIG["ts_prediction_length"],
        presets=CONFIG["ts_presets"],
        force_retrain=CONFIG["force_ts_retrain"],
    )
    
    # Generate forecasts
    forecast_cache_path = paths["processed"] / "forecasts" / symbol / f"fc_{CONFIG['feature_set']}_{RUN_ID}.parquet"
    forecast_cache_path.parent.mkdir(parents=True, exist_ok=True)
    
    forecasts = load_or_generate_forecasts(
        predictor=ts_predictor,
        bars_df=symbol_bars,
        symbol=symbol,
        decision_timestamps=decision_timestamps,
        cache_path=forecast_cache_path,
        horizons=horizons,
        force_regenerate=CONFIG["force_forecast_regenerate"],
    )
    
    print(f"Forecasts shape: {forecasts.shape}")
    all_forecasts[symbol] = forecasts

print("\nForecast generation complete!")

## 3. Merge All Features

In [ ]:
# Merge all features for each target symbol
merged_dfs = {}

for symbol in CONFIG["symbols_to_train"]:
    print(f"\nMerging features for {symbol}...")
    
    # Get labeled data
    if isinstance(full_df.index, pd.MultiIndex):
        symbol_df = full_df.loc[symbol].copy()
    else:
        symbol_df = full_df[full_df["symbol"] == symbol].copy()
    symbol_df = symbol_df.sort_index()
    
    max_rows = CONFIG["max_rows_per_symbol"]
    if max_rows and len(symbol_df) > max_rows:
        symbol_df = symbol_df.iloc[-max_rows:]
    
    # 1. Merge asset's own forecast features
    merged = merge_forecast_features(
        tabular_df=symbol_df,
        forecasts_df=all_forecasts[symbol],
        feature_set=CONFIG["feature_set"],
        prefix=CONFIG["forecast_prefix"],
    )
    print(f"  After own forecast features: {len(merged.columns)} columns")
    
    # 2. Add context features
    context_forecasts = {ctx: all_forecasts[ctx] for ctx in CONFIG["context_symbols"]}
    merged = add_context_features(
        df=merged,
        context_forecasts=context_forecasts,
        target_symbol=symbol,
        context_symbols=CONFIG["context_symbols"],
        feature_set=CONFIG["feature_set"],
        include_relative=CONFIG["include_relative"],
    )
    print(f"  After context features: {len(merged.columns)} columns")
    
    merged_dfs[symbol] = merged

print("\nFeature merging complete!")

In [ ]:
# Define all feature columns
baseline_features = get_feature_columns(True)
forecast_features = get_forecast_feature_names(CONFIG["feature_set"], CONFIG["forecast_prefix"])

# Context features
context_features = []
for ctx_sym in CONFIG["context_symbols"]:
    prefix = f"ctx_{ctx_sym.lower()}_"
    for feat in feature_set_config["features"]:
        for h in horizons:
            context_features.append(f"{prefix}{feat}_{h}")

# Relative features
relative_features = []
if CONFIG["include_relative"]:
    for ctx_sym in CONFIG["context_symbols"]:
        for h in horizons:
            relative_features.append(f"rel_{ctx_sym.lower()}_mu_{h}")
            relative_features.append(f"rel_{ctx_sym.lower()}_unc_{h}")

all_feature_cols = baseline_features + forecast_features + context_features + relative_features

print(f"Feature breakdown:")
print(f"  Baseline: {len(baseline_features)}")
print(f"  Own forecast: {len(forecast_features)}")
print(f"  Context: {len(context_features)}")
print(f"  Relative: {len(relative_features)}")
print(f"  TOTAL: {len(all_feature_cols)}")

## 4. Train Tabular Model

In [ ]:
# Store results
all_results = {}

for symbol in CONFIG["symbols_to_train"]:
    print(f"\n{'=' * 60}")
    print(f"TRAINING MODEL FOR: {symbol}")
    print(f"{'=' * 60}")
    
    symbol_df = merged_dfs[symbol].copy()
    
    # Filter to available features
    available_features = [f for f in all_feature_cols if f in symbol_df.columns]
    missing_features = [f for f in all_feature_cols if f not in symbol_df.columns]
    if missing_features:
        print(f"Warning: Missing features: {missing_features[:5]}...")
    
    # Clean data
    required_cols = available_features + [CONFIG["label_col"]]
    symbol_df_clean = symbol_df.dropna(subset=required_cols).copy()
    
    print(f"\nData after cleaning: {len(symbol_df_clean)} rows (dropped {len(symbol_df) - len(symbol_df_clean)})")
    
    symbol_df_clean = symbol_df_clean.reset_index(drop=False)
    if "timestamp" not in symbol_df_clean.columns and "index" in symbol_df_clean.columns:
        symbol_df_clean = symbol_df_clean.rename(columns={"index": "timestamp"})
    
    # Create split
    try:
        split = create_single_split(
            n_samples=len(symbol_df_clean),
            vertical_barrier_bars=CONFIG["vertical_barrier_bars"],
            embargo_bars=CONFIG["embargo_bars"],
            tune_window=CONFIG["tune_window"],
            test_window=CONFIG["test_window"],
            min_train_size=CONFIG["min_train_size"],
        )
    except ValueError as e:
        print(f"ERROR: {e}")
        continue
    
    is_valid = validate_split_no_leakage(split, CONFIG["vertical_barrier_bars"])
    print(f"Split valid: {is_valid}")
    
    train_df, tune_df, test_df = apply_split_to_dataframe(symbol_df_clean, split)
    print(f"Split: Train={len(train_df)}, Tune={len(tune_df)}, Test={len(test_df)}")
    
    # Train
    model_path = paths["models"] / "exp3" / symbol / RUN_ID
    model_path.mkdir(parents=True, exist_ok=True)
    
    print(f"\nTraining with {len(available_features)} features...")
    
    predictor = train_tabular_baseline(
        train_df=train_df,
        tune_df=tune_df,
        feature_cols=available_features,
        label_col=CONFIG["label_col"],
        model_path=model_path,
        time_limit=CONFIG["time_limit_sec"],
        presets=CONFIG["presets"],
        random_seed=CONFIG["random_seed"],
        verbosity=2,
    )
    print("Training complete!")
    
    # Predictions
    predictions_df = predict_tabular(predictor, test_df, available_features)
    predictions_df["actual_label"] = test_df[CONFIG["label_col"]].values
    if "timestamp" in test_df.columns:
        predictions_df["timestamp"] = test_df["timestamp"].values
    
    run_dir = paths["runs"] / f"exp3_{RUN_ID}"
    run_dir.mkdir(parents=True, exist_ok=True)
    predictions_path = run_dir / f"predictions_{symbol}.parquet"
    predictions_df.to_parquet(predictions_path)
    
    # Metrics
    metrics = compute_all_metrics(
        y_true=test_df[CONFIG["label_col"]],
        y_pred=predictions_df["predicted_label"],
        y_train=train_df[CONFIG["label_col"]],
        y_tune=tune_df[CONFIG["label_col"]],
    )
    
    metrics["run_info"] = {
        "run_id": RUN_ID,
        "experiment": "exp3",
        "symbol": symbol,
        "timestamp": datetime.now().isoformat(),
        "config": CONFIG,
        "feature_count": len(available_features),
        "context_symbols": CONFIG["context_symbols"],
    }
    
    metrics_path = run_dir / f"metrics_{symbol}.json"
    save_metrics(metrics, metrics_path)
    
    print_metrics_summary(metrics)
    
    all_results[symbol] = {
        "metrics": metrics,
        "predictor": predictor,
        "model_path": model_path,
    }

print(f"\n{'=' * 60}")
print("ALL SYMBOLS COMPLETE!")
print(f"{'=' * 60}")

## 5. Summary

In [ ]:
print(f"\n{'=' * 60}")
print("EXPERIMENT 3 SUMMARY")
print(f"{'=' * 60}")

print(f"\nRun ID: {RUN_ID}")
print(f"Symbols trained: {list(all_results.keys())}")
print(f"Context symbols: {CONFIG['context_symbols']}")
print(f"Feature set: {CONFIG['feature_set']}")
print(f"Total features: {len(all_feature_cols)}")

print(f"\nPerformance Summary:")
print("-" * 50)
print(f"{'Symbol':<10} {'Accuracy':>10} {'Bal Acc':>10} {'Macro F1':>10}")
print("-" * 50)

for symbol, result in all_results.items():
    m = result["metrics"]["classification"]
    print(f"{symbol:<10} {m['accuracy']:>10.4f} {m['balanced_accuracy']:>10.4f} {m['macro_f1']:>10.4f}")

In [ ]:
# Compare with previous experiments
print("\nComparison with Previous Experiments:")
print("-" * 60)

for exp_name in ["exp1", "exp2"]:
    exp_runs = list((paths["runs"]).glob(f"{exp_name}_*"))
    if exp_runs:
        latest_run = sorted(exp_runs)[-1]
        print(f"\n{exp_name.upper()} (latest: {latest_run.name}):")
        
        for symbol in all_results.keys():
            prev_metrics_path = latest_run / f"metrics_{symbol}.json"
            if prev_metrics_path.exists():
                with open(prev_metrics_path) as f:
                    prev_metrics = json.load(f)
                
                prev_ba = prev_metrics["classification"]["balanced_accuracy"]
                curr_ba = all_results[symbol]["metrics"]["classification"]["balanced_accuracy"]
                diff = curr_ba - prev_ba
                
                print(f"  {symbol}: {prev_ba:.4f} -> {curr_ba:.4f} ({diff:+.4f})")

---

**Experiment 3 Complete!**

Next: Experiment 4 adds forecast error/coverage features (model health monitoring).